# Old ways of using OpenAI API

## via openai python ChatCompletion class

In [3]:
import os
import openai
openai.api_key = os.getenv("OPENAI_API_KEY")

completion=openai.ChatCompletion.create(
    model = "gpt-3.5-turbo",
    messages = [
        {"role":"system","content":"You are a helpful assistant."},
        {"role":'user',"content":"Hello"}
               
               ]
)

print(completion.choices[0].message)

{
  "role": "assistant",
  "content": "Hello! How can I assist you today?"
}


## Via CURL command

```bash
curl https://api.openai.com/v1/chat/completions \
  -H "Content-Type: application/json" \
  -H "Authorization: Bearer $OPENAI_API_KEY" \
  -d '{
    "model": "gpt-3.5-turbo",
    "messages": [
      {"role": "system", "content": "You are a helpful assistant."}, 
      {"role": "user", "content": "Hello!"}
      ]
  }'
```

## Via request module

```python
OPENAI_API_KEY=openai.api_key = os.getenv("OPENAI_API_KEY")


import requests

headers  ={

    "Contenet-Type" : "application/json",
    "Authorization" : "Bearer " + OPENAI_API_KEY
}
json_data = {
    'model':'gpt-3.5-turbo',
    'messages': [
        {'role': 'system', 'content': 'You are ahelpful assistant.' },
        {'role': 'user', 'content': 'Hello!' },
    ]
}
response = requests.post(
    "https://api.openai.com/v1/chat/completions",
    headers=headers,
    json=json_data
 )

```


In [9]:
# byrunning the above code the response we get
response.json()

{'id': 'chatcmpl-7Y7x5F2QXbfEvxe1ooi2KVBQm39Bq',
 'object': 'chat.completion',
 'created': 1688368075,
 'model': 'gpt-3.5-turbo-0613',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': 'Hello! How can I assist you today?'},
   'finish_reason': 'stop'}],
 'usage': {'prompt_tokens': 20, 'completion_tokens': 9, 'total_tokens': 29}}

# Weather Chatbot


## Checkout the following example - 

```bash
curl https://api.openai.com/v1/chat/completions -u :$OPENAI_API_KEY -H 'Content-Type: application/json' -d '{
  "model": "gpt-3.5-turbo-0613",
  "messages": [
    {"role": "user", "content": "What is the weather like in Boston?"}
  ],
  "functions": [
    {
      "name": "get_current_weather",
      "description": "Get the current weather in a given location",
      "parameters": {
        "type": "object",
        "properties": {
          "location": {
            "type": "string",
            "description": "The city and state, e.g. San Francisco, CA"
          },
          "unit": {
            "type": "string",
            "enum": ["celsius", "fahrenheit"]
          }
        },
        "required": ["location"]
      }
    }
  ]
}'
```

focus on the function keyword and the details under it can be found in a functions docstring. So from the above function description,
you can assume the `get_current_weather` as follows - 

```python
def get_current_weather(location, unit):
    """Get the current weather in a given location

    Args:
        location (str): The city and state, e.g. San Francisco, CA
        unit (str): ["celsius", "fahrenheit"]
    """
    pass
```

In [ ]:
import requests
import json
import os
from dotenv import load_dotenv
load_dotenv()

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
GPT_MODEL = 'gpt-3.5-turbo-0613' #new model for function calling abilities



In [26]:
import requests

def chat_completion_request(messages, functions=None, function_call=None, model=GPT_MODEL):
    headers = {
        'Content_Type': 'application/Json',
        'Authorization': 'Bearer ' + OPENAI_API_KEY,
    }
    json_data ={
        "model":model,
        "messages":messages
    }
    if functions is not None:
        json_data.update({"functions":functions})
    if function_call is not None:
        json_data.update({"fuction_call":function_call})

    try:
        response=requests.post(
            "https://api.openai.com/v1/chat/completions",
            headers = headers,
            json = json_data
        )
        return response
    except Exception as e:
        print(f"\n>>>Unable to generate ChatCompletion response")
        print(f"\n>>> Exception: {e}")
        raise e




In [27]:
messages = [
    {'role':'system','content':"you are a helpful assistant"},
    {'role':'user','content':'Hi, there'},
]

chat_response = chat_completion_request(messages=messages)

print(chat_response)
chat_response.json()


<Response [200]>


{'id': 'chatcmpl-7Y94bMvRpY8QvFF1Y49hl5J0umpWi',
 'object': 'chat.completion',
 'created': 1688372385,
 'model': 'gpt-3.5-turbo-0613',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': 'Hello! How can I assist you today?'},
   'finish_reason': 'stop'}],
 'usage': {'prompt_tokens': 19, 'completion_tokens': 9, 'total_tokens': 28}}

In [31]:
assistant_message = chat_response.json()
assistant_message['choices'][0]['message']['content']

'Hello! How can I assist you today?'

In [32]:
creator_name = "mach-42"

system_instruction = f"""
You are Weather bot created by {creator_name}, \
You first greet the customer, and you MUST introduce yourself as -
"Hi I am Weather bot created by {creator_name}! How may I assist you today?" \
an automated service to tell Weather infomation \
based on the location provided by the user. \
then collects the location information\
Don't make assumptions about what \
values to plug into functions. \
Ask for clarification if a user request is ambiguous.
You respond in a short, very conversational friendly style. \
"""

In [33]:
messages = [{'role':'system',"content":system_instruction}]

In [34]:
functions = [
    {
        "name":"get_current_weather", # Name of the function
        "description":"This weather function will fetch the current weather report",
        'parameters':{
            "type":"object",
            'properties':{ # 1st arguement
                'location':{ #arguement name
                    'type':'string', #arguement type
                    'description':'The city and state, eg- new Delgi, Bengluru' # description of a function with example
                }
            },
            'required':['location'] # required requirement i.e ist arg is required for this function
        }


    }
]

In [35]:
def get_current_weather(location):
    RAPID_API_KEY = os.getenv('RAPID_API_KEY')
    try:

        url = "https://weatherapi-com.p.rapidapi.com/current.json"

        querystring = {"q":location}
        print(f"\n>>>> got the querystring as: {querystring}")


        headers = {
            "X-RapidAPI-Key": RAPID_API_KEY,
            "X-RapidAPI-Host": "weatherapi-com.p.rapidapi.com"
        }

        response = requests.get(url, headers=headers, params=querystring)
        response_json = response.json()
        print(f"\n>>>> got the RAPID API response as: \n{response_json}")
        return response_json

    except Exception as e:
        print("Exception occured in fetching rapid api as : {e}")
        raise e
    

In [36]:
user_message = "Hi there!"
messages.append({"role": "user", "content": user_message})
chat_response = chat_completion_request(messages = messages, functions = functions)
chat_response.json()

{'id': 'chatcmpl-7Y9XVXWHxlayLPLfm3ijyrooPnZ6q',
 'object': 'chat.completion',
 'created': 1688374177,
 'model': 'gpt-3.5-turbo-0613',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': 'Hi there! How may I assist you today? I am Weather bot created by mach-42!'},
   'finish_reason': 'stop'}],
 'usage': {'prompt_tokens': 170, 'completion_tokens': 21, 'total_tokens': 191}}